# GitHub link

# import libraries

In [1]:
import nltk
import numpy as np
import pandas as pd

from nltk.stem import WordNetLemmatizer
from sklearn.linear_model import LogisticRegression
from bs4 import BeautifulSoup
from sklearn.naive_bayes import MultinomialNB

# Read Data

In [2]:
#Read the Data
news_data = pd.read_csv("fake_or_real_news.csv")

In [3]:
news_data.head()

,Unnamed: 0,title,text,label
0,8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL


In [4]:
# split the data into X and Y
X = news_data.text
y = news_data.label

In [5]:
#split the data in test and training set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.33, random_state=53)

In [6]:
from sklearn.feature_extraction.text import CountVectorizer
count_vectorizer = CountVectorizer(stop_words='english')

In [7]:
count_train = count_vectorizer.fit_transform(X_train)

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(stop_words='english',max_df=0.7)

# Method to clean the text

In [9]:
import string
from nltk.corpus import stopwords
StopWords = stopwords.words("english")
def clean_message(mess):
    """
    Remove all punctuation and stopwords
    """
    #Filter out the punctuation
    nopunc = [char for char in mess if char not in string.punctuation]
    # Join the characters again to form the string.
    nopunc = ''.join(nopunc)
    #remove stopwords
    return [word for word in nopunc.split() if word.lower() not in StopWords]

In [10]:
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
pipeline = Pipeline([
    ('bow', CountVectorizer(analyzer=clean_message)),
    ('tfidf', TfidfTransformer()),
    ('classifier', MultinomialNB())])

In [11]:
pipeline.fit(X_train,y_train)

Pipeline(memory=None,
     steps=[('bow', CountVectorizer(analyzer=<function clean_message at 0x000000000BF9E1E0>,
        binary=False, decode_error='strict', dtype=<class 'numpy.int64'>,
        encoding='utf-8', input='content', lowercase=True, max_df=1.0,
        max_features=None, min_df=1, ngram_range=(1, 1), preprocess...f=False, use_idf=True)), ('classifier', MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))])

In [12]:
predictions = pipeline.predict(X_test)

In [14]:
from sklearn.metrics import classification_report
print(classification_report(predictions,y_test))

             precision    recall  f1-score   support

       FAKE       0.65      0.99      0.79       670
       REAL       0.99      0.76      0.86      1421

avg / total       0.88      0.83      0.83      2091

